# Deep Learning: I

Prima di parlare di *neural networks*, introduciamo alcuni concetti.

### Loss Functions

Una **loss function** è una specie di *cost function*, o una *likelihood*, o una *optimization function*, o una *objective function*.

Sto cercando di minimizzare l'offset tra un modello e dei dati. La differenza è che la loss function è valutata su un singolo training example e non su tutto il data set!

Tipicamente, si usano **L2 Loss Functions**: ad esempio, il $\chi^{2}$ è una L2 loss function; la *Huber Loss Function* una **L1 Loss Function**, che è più resistente agli outplier.

Per la classification si plotta LOSS vs. y$\times$f(x), dove f(x) corrisponde alla classe nota (=+1 o -1). Se il prodotto è positivo, prevedo +1. Se è negativo, -1. Definisco la LOSS = 0 per y * f(x) = 1, ovvero quando ho beccato la risposta giusta.

Guarda il notebook per vedere cosa succede alla loss in funzione di y * f(x). Come può essere anche più grande di 1?

Ok, mi serve una Loss Function che abbia senso per la classification.

1. Zero-One Loss: aumenti la loss function di 1 ogni volta che fai una wrong prediction
    - y f(x) < 0 => 1
    - y f(x) > 0 => 0
    È hard to minimize!
    
Provo con qualcos'altro:

2. Hinge Loss: max(0,1-y * f(x)) // no contribution to the loss for values $\geq 1$, ma linearly increasing loss per smaller values -> penalizzo sia le prediction sbagliate sia le predizioni corrette che hanno LOW confidence.

3. Logistic Loss (log loss, cross entropy loss): similar properties, ma è smoother e ha less and less penalty for more and more confident +1 predictions


### Gradient Descent

Sto cercando l'optimal extremal position of a cost function. Fin qui ho cercato di trovare i model parameters $\theta$ che minimizzano O l'errore di regressione O l'errore di classificazione, quando fitto i dati.

- Ogni tanto ho la soluzione analitica di $\theta$
- Con MCMC, faccio un sampling del parameter space e cerco la risposta migliore
- Che succede se "sei in cima ad una montagna e NON sai da che parte si scende?" -> Inizi ad andare "in giù" -> **gradient descent**

> Cerco di determinare il LOCAL GRADIENT della loss function rispetto a $\theta$ e vado nella STEEPEST direction, finché il gradiente è zero.

Vedi notebook. Comunque esce un $\eta$, che chiamo **learning rate**, che determina quanto grossi sono i miei "step down": se lo step size è troppo piccolo, ci vuole troppo a convergere. Se lo step è troppo lungo, rischi di perdere il "bottom", e può darsi che la soluzione diverga.

### Cost

Attenzione anche a non finire nel minimo locale al posto del minimo globale! Un grosso pro delle funzioni L2 è che c'è UN minimo globale. 

Esempio sul notebook: $\eta$ troppo basso, giusto, troppo alto.

### AdaBoost

Guarda il notebook per vedere cosa fa e cosa succede se cambi la $\eta$.

# Neural Networks

Il termine "neural network" è overused, e riguarda una gran varietà di deep learning approaches. Noi parliamo di **multi-layer perceptron**.

Immagine nel notebook: a sinistra ho FEATURES/ATTRIBUTES dei miei input data. In centro ho i NEURONI, che prendono le informazioni dall'input e decidono se "accendersi" o meno in base a qualche criterio. Il loro layer si chiama "hidden layer". A destra ho i risultati dei neuroni, l'OUTPUT. Le linee che connettono input, neuroni ed output si chiamano SINAPSI.

Il lavoro di una sinapsi è quello di prendere gli input values, moltiplicarli per un peso $w$ e di aggiungere un bias $b$, prima di passarli al neurone.

$z = \sum_{i} wx_{i} +b$ -> il bias determina l'input level che "accende" il neurone. È sempre presente, ed è unico per ciascun neurone, ma teniamolo a zero per semplicità.

Il neurone somma gli input da tutte le sinapsi ad esso connesse, ed applica la **activation function**, ad esempio una sigmoide $a = [1+exp(-z)]^{-1}$

> La rete neurale impara i pesi e i bias delle sinapsi che sono richieste per produrre un modello accurato di $y_{train}$. (Giusto?!)

Pensala in forma matriciale: $Z^{(2)} = X W^{(1)}$

Se D è il numero di attributi, ed H è il numero di neuroni nell'hidden layer, allora X è una matrice NxD, mentre $W^{(1)}$ è una DxH. Il risultato allora è una matrice NxH.

Quando applico l'activation function ad ogni entry di $Z^{(2)}$, ho $A^{(2)} = f(Z^{(2)})$. 

Questi valori sono gli input per il prossimo set di sinapsi: $Z^{(3)} = A^{(2)} W^{(2)}$, dove stavolta ho il prodotto tra una matrice HxO e mi esce una NxO.

Un'altra activation function che posso usare è applicata a $Z^{(3)}$ e mi dà $\hat{y}$, il mio estimatore di y.

---

Esempio: supponiamo di avere 100 persone di peso e altezza note, di cui ho misurato la "size" di scarpe, cintura, cappello.

Voglio prevedere altezza e peso delle persone, di cui so solo queste misure: la neural network essenzialmente determina peso e bias delle sinapsi, che di solito inizializzo a caso. Lo faccio, minimizzando la cost function, che confronta i veri valori di y con quelli predetti. 

Ok, se ho UN peso e voglio controllare 1000 valori, non male: se invece ne ho VENTI, mi ritrovo a 20^1000 possibili combinazioni!

### Backpropagation

Ci sono alcune tecniche per salvarsi dal curse of dimensionalilty. Posso scrivere una FORMULA ANALITICA per il GRADIENTE, andando all'indietro nella mia rete neurale. Posso quindi usarla, per aggiornare i miei pesi e bias.

Vedi notebook: VOGLIO UNA FUNZIONE DIFFERENZIABILE. Le voglio anche calcolare separatamente: in questo modo posso fare **backpropagation** delle error contributions along each neuron. Ok, alla fine arrivo a sapere quale sia la direzione per il downhill? Molto bene.

**NON HO CAPITO**.

---

## A computational digression

Deep learning => calcolo un sacco di derivate. Calcolarle, dal punto di vista numerico, è difficile. Vuoi scriverla col limite del rapporto incrementale? NUMERICALLY UNSTABLE. Ci sono metodi migliori, ma rimane comunque un problema anche se mi fermo ad ordini superiori dello sviluppo.

Perché allora il deep learning funziona? La risposta sta nell'**automatic differentiation**. L'dea di base è dare in pasto alla macchina una lista di funzioni elemetari, e il calcolo della derivata diventa una applicazione della chain rule. (**EH!?**) In pratica, mi riduco a calcolare derivate di qualsiasi ordine molto velocemente e con "working precision".

La risposta sta anche nell'exploitation delle GPU al posto delle CPU. 

---

Usiamo il multi-layer perceptron classifier sul Boston House Price dataset: vedi notebook.

### Guidelines: Number of Layers

> For data that can be represented by a linear model, no layers are required (McCullagh & Nelder 1989). A single layer network can approximate any continuous function. Two layers can represent arbitrary decision boundaries for smooth functions (Lippmann 1987). More layers can represent non-continuous or complex structure within the data.

### Guidelines: Number of Neurons

* Tipicamente si sceglie un numero che va da due volte il numero di input nodes e un numero tra il numero di input e output nodes.

* Se ci sono TANTI hidden layers (definisci "tanti"?) allora parlo di **deep neural networ** o **deep learning**.

* Ogni tanto il numero di neuroni in each layer va giù, ma può essere comodo averne lo stesso numero per avere UN hyperparameter (il numero di neuroni) e non uno diverso per hidden layer

* In pratica, un approccio ragionevole è specificare più layer e neurons del necessario, e fare regularization -> fermo il training quando la cross-validation error raggiunge un minimo, e parlo di *early stopping*.

### Guidelines: Activation Functions

"Quanto segnale serve per "attivare" un neurone"? -> The more signal, the more likely the neuron will fire.

Guarda il notebook per guardare le activation functions. La binary function non va tanto bene, è meglio una sigmoide. In particolare, la sigmoide è differenziabile, e la cosa ti piace assai per fare la backpropagation!

**Un altro aspetto importante delle NON-linear activation functions è che sono ciò che permette alle neural networks di risolvere problemi NON lineari! Se usassi solo funzioni lineari, potrei risolvere solo problemi lineari.**

*Guarda il notebook per il problema di vanishing and exploding gradients.* La ReLU activation function è una possibile alternativa alla sigmoide. 

> Usa la sigmoide // softmax per output di classification <br> Usa la ReLU per layers che non sono quelli di output

---

### Regularization

Posso usare le solite tecniche (LASSO//RIDGE), ma posso usare anche la **dropout**: "butto via" alcuni neuroni durante il training. L'analogo è quello di un'azienda che deve funzionare anche se alcuni dei suoi dipendenti sono in malattia -> rinforzi la rete.

### Batch Normalization

Stabilizza il processo, e lo velocizza. NON HO IDEA DI COSA FACCIA, LO VEDIAMO LA PROSSIMA VOLTA.

### Data Augmentation

A volte trasformare i dati dà un migliore fit -> **data augmentation**. Un esempio sta nel ruotare le immagini disponibili per produrre more training data -> la network neurale impara a riconoscere le "truly important" features.

### Faster Optimizers

Non ne parliamo, ma prova a cercare il migliore da usare facendo cross-validation!

---

# Mind your set: training, validation, and test

Deep learning implementations => decine di iperparametri!

Having a test set can save me from SUBTLE overfitting.

- I parametri sono inferiti dal training set
- La performance è valutata sul validation set -> tune the hyperparameters
- Valuto l'overall performance su un external test set -> **Analysis Unbinding**